In [22]:
import sys
sys.path.append('/home/sensei/jupy-notebooks/Analytics/PorterFarms/')

print("============================================")
print("/  RfSurvey is running.            /")
print("============================================")

import requests
from datetime import datetime, timedelta
import pytz
from slackclient import SlackClient
import json
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import configparser

import geopy
from geopy.distance import geodesic

config = configparser.ConfigParser()
config.read("../../analytics_secrets.ini")

_ACTIVE_STANDBY = config['DEFAULT']['role']
if _ACTIVE_STANDBY == 'STANDBY':
    print("STANDBY")
    raise SystemExit("Stop right there!")
else:
  _SLACK_TOKEN = config['slack']['token']
  _CHIRPSTACK_USER = config['chirpstack']['user']
  _CHIRPSTACK_PASS = config['chirpstack']['password']
  _DB_HOST  = config['kanjidb']['dbhost']
  _DB_PORT  = config['kanjidb']['dbport']
  _DB_NAME  = config['kanjidb']['dbname']
  _DB_USER  = config['kanjidb']['dbuser']
  _DB_PASS  = config['kanjidb']['dbpass']

  _SLACK_RFSURVEYCHANNEL_NAME = config['analytics']['rfsurveychannelname']
  _USE_DROPBOX   = config['dropbox']['usedropbox']
  _FALLBACK_IMAGE = config['dropbox']['fallbackimage']

  _LOG_DEBUG = 0
  _LOG_INFO  = 1
  _LOG_ERROR = 2
  _LOG_LEVEL = int(config['DEFAULT']['loglevel'])

def logger(level, message):
   if level >= _LOG_LEVEL:
      print(message)

logger(_LOG_DEBUG, "{} {} {} {} {}".format(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS))

import kanjiticketing as kt

conn = kt.getKanjiDbConnection(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS)
if conn is not None:
  logger(_LOG_INFO, "Welcome to Jupyter Notebook.  You are connected to the Kanji database!")
else:
  logger(_LOG_ERROR, "You are not connected to the database.")

messagetemplate = "[\
   {\"type\": \"section\", \
		\"text\": { \
			\"type\": \"mrkdwn\", \
			\"text\": \"*<fakeLink.toUserProfiles.com|Iris / Zelda 1-1>*\\nTuesday, January 21 4:00-4:30pm\\nBuilding 2 - Havarti Cheese (3)\\n2 guests\" \
		}, \
		\"accessory\": { \
			\"type\": \"image\", \
			\"image_url\": \"https://api.slack.com/img/blocks/bkb_template_images/notifications.png\", \
			\"alt_text\": \"calendar thumbnail\" \
		} \
   } ]"

/  RfSurvey is running.            /
Welcome to Jupyter Notebook.  You are connected to the Kanji database!


In [23]:
def postMessageToSlack(blockmessage):    
    sc = SlackClient(_SLACK_TOKEN)
    slackchannel = _SLACK_RFSURVEYCHANNEL_NAME
    response = sc.api_call("chat.postMessage", channel=slackchannel, blocks=blockmessage)
    print(blockmessage) 
    if not 'ok' in response or not response['ok']:
      logger(_LOG_ERROR, "Error posting message to Slack channel")
      logger(_LOG_ERROR, blockmessage)
      logger(_LOG_ERROR, response)
    else:
      logger(_LOG_INFO, "Ok posting message to Slack channel")    

In [24]:
_LOG_LEVEL = _LOG_INFO

_PREFERRED_IMAGE = "https://www.dropbox.com/s/gft8wgriqjvma04/lora-snr.jpg?raw=1"

if _USE_DROPBOX == 'true':
  locationimageurl = _PREFERRED_IMAGE
else:
  locationimageurl = _FALLBACK_IMAGE

#Mote types
_MAINS_MOTE_TYPE = 10000
_BATTERY_MOTE_TYPE = 10002

now = datetime.now(pytz.utc)  #tz Aware
logger(_LOG_DEBUG, "Current time is {}".format(now))

ticketnow = datetime.now(pytz.utc)  #tz Aware
now = datetime.now() + timedelta(hours = 4)

_NODE_SENSORTYPE = 40           # the GEOLOCATION type

motequery = "SELECT * FROM kanji_node WHERE deploystate_id=10001 AND (nodetype_id={} OR nodetype_id={})".format(_MAINS_MOTE_TYPE, _BATTERY_MOTE_TYPE)
logger(_LOG_DEBUG, motequery)
df = pd.read_sql(motequery, conn)

logger(_LOG_DEBUG, "number of motes {}".format(len(df.index)))
for ind in df.index:
    nodename = df['name'][ind]
    node_id = df['idnode'][ind]
    #eventquery = "SELECT node.name AS gatewayname,\
    #              location.description AS location, \
    #              * FROM kanji_eventlog event \
    #              JOIN kanji_node node ON (node.name LIKE concat('%',event.gateway,'%')) \
    #              JOIN kanji_location location ON location.idlocation=node.location_id \
    #              JOIN kanji_customer customer ON location.customer_id=customer.idcustomer \
    #              JOIN kanji_slackchannel slackchannel ON customer.slacksystemticketchannel_id=slackchannel.idslackchannel \
    #              WHERE timestamp> NOW() - INTERVAL '1 MINUTE' AND node_id={} AND sensortype_id={} ORDER BY timestamp desc;".format(node_id, _NODE_SENSORTYPE)
    eventquery = "SELECT node.idnode, location.description AS location, \
                  event.timestamp, event.eventdata, \
                  event.lorasnr, event.rssi, event.gateway FROM kanji_eventlog event \
                  JOIN kanji_node node on node.idnode=event.node_id \
                  JOIN kanji_location location ON location.idlocation=node.location_id \
                  WHERE timestamp> NOW() - INTERVAL '2 MINUTES' \
                  AND node_id={} \
                  AND sensortype_id={} ORDER BY timestamp desc LIMIT 1".format(node_id, _NODE_SENSORTYPE)
    logger(_LOG_DEBUG, eventquery)
    df2 = pd.read_sql(eventquery, conn)
    if len(df2.index)>0:      
      for ind2 in df2.index:
        timestamp = df2['timestamp'][ind2]
        eventdata = df2['eventdata'][ind2]        
        eventjson = json.loads(eventdata.replace("'","\""))
        logger(_LOG_DEBUG, eventjson)
        logger(_LOG_DEBUG, eventjson['popdata'])
        latitude = float(eventjson['popdata'][0])
        longitude = float(eventjson['popdata'][1])
        altitude = float(eventjson['popdata'][2])
        marcopolo = float(eventjson['popdata'][3])
        logger(_LOG_DEBUG, "Polo={}".format(marcopolo))
        
        lorasnr = df2['lorasnr'][ind2]
        rssi = df2['rssi'][ind2]
        gw_name = df2['gateway'][ind2]
        motelocation = df2['location'][ind2]        
        gatewayquery = "SELECT * FROM kanji_node node JOIN kanji_location location ON location.idlocation=node.location_id \
                        WHERE name='{}' LIMIT 1".format(gw_name)
        df3 = pd.read_sql(gatewayquery, conn)
        if len(df3.index)>0:
          logger(_LOG_DEBUG, "gatewayquery ={}".format(gatewayquery))
          gw_latitude = df3['latitude'][0]
          gw_longitude = df3['longitude'][0]
          logger(_LOG_INFO, "{}\t{}\tstar1\t{}\t{}".format(gw_latitude, gw_longitude, 'black', gw_name))
                 
          nodelocation = (latitude, longitude)
          gatewaylocation = (gw_latitude, gw_longitude)
          loradistance = (float) (geodesic(nodelocation, gatewaylocation).miles * 5280/3)
          if rssi < -120.0:
            color = 'red'
          elif rssi < -100.0:
            color = 'yellow'
          else:
            color = 'green'
          if loradistance>0.0:
            logger(_LOG_INFO, "{}\t{}\tcircle1\t{}\t{}\t{} dist={:<5.1f}yd snr={}dB, rssi={}dB".format(latitude, longitude, color, ind2, gw_name, loradistance, lorasnr, rssi))
          geomessage = "{} *{} gateway reports:*\n*{}/{} @ {:<5.1f}yards downrange.*\n*SNR={}dB, RSSI={}dB*" \
            .format(timestamp.strftime("%m/%d, %H:%M:%S"), gw_name, nodename, motelocation, loradistance, lorasnr, rssi)
          logger(_LOG_INFO, geomessage)
          blockmessage = json.loads(messagetemplate)
          blockmessage[0]["accessory"]["image_url"] = locationimageurl
          blockmessage[0]["text"]["text"] = "{}".format(geomessage)
        
          if marcopolo == 0.0:
            logger(_LOG_DEBUG, "Posting message to Slack")
            postMessageToSlack(blockmessage)
          else:
            logger(_LOG_DEBUG, "Polo bit not set")
        else:
            logger(_LOG_DEBUG, "Gateway query returned null")
logger(_LOG_INFO, "RfSurvey Done!") 

35.27	-80.65	star1	black	Sherman
06/10, 14:37:15 *Sherman gateway reports:*
*agMote-20000/Mobile Unit @ 0.0  yards downrange.*
*SNR=8.0dB, RSSI=-85dB*
RfSurvey Done!
